# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
df = pd.read_csv('../output_data/Cities_Weather_Data.csv')
df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

FigureLayout={
    'width': '600x',
    'height': '600px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=FigureLayout, zoom_level=2, center=(0,0))

In [ ]:
locations=df[["Lat","Lng"]].astype(float)
humidity=df["Humidity"].astype(float)

# Create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Ideal Weather Conditions
# Max temp lower than 75 degrees but higher than 70
# Wind speed less than 5 mph
# Humidity less than 80% but higher than 40%

ideal_df=df.loc[(df["Max Temp"]>=70)&(df["Max Temp"]<75)&(df["Wind Speed"]<5)&(df["Humidity"]<80)&(df["Humidity"]>40)&(df["Cloudiness"]<20)]

if len(ideal_df)>10:
    ideal_df=ideal_df.iloc[:10,:]
ideal_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df=ideal_df.copy()
hname=[]

target_locations=[]
target_radius=5000
target_query="hotel"
target_type="lodging"

x=range(len(ideal_df))

#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
for n in x:
    coord=f"{ideal_df.iloc[n,2]}, {ideal_df.iloc[n,3]}"
    search_params = {
        "location":coord,
        "type":target_type,
        "radius": target_radius,
        "key":g_key
    }
    response = requests.get(base_url, params=search_params).json()
    hname.append(response["results"][0]["name"])
hotel_df["Hotel Name"]=hname


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
hotel_df
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig